### **SR Basado en Filtros Colaborativos**

Voy a necesitar en un archivo el: 
- ID usuarios, ID items, rating correspondiente

Para eso voy a importar el archivo **review.json** y me voy a quedar con las columnas:
- user_id, business_id y stars

El modelo es del tipo Model Based que no se basa solo en interacciones sino que usa algoritmos por detras. Se va a probar en esta notebook el **SVD** que es un enfoque de factorización matricial que modela características latentes compartidas tanto por los usuarios como por los ítems."Predice las valoraciones del usuario para cada restaurante"

El archivo que leo ya es un parquet filtrado para que no haya dos de la combinacion Usuario Restaurante

#### Imports y Lectura del Archivo

In [24]:
pip install chart_studio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip setuptools wheel

In [ ]:
pip install --upgrade pandas

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import datapane as dp
#import chart_studio.plotly as py
import chart_studio
import os

from sklearn.model_selection import train_test_split as skl_train_test_split
from surprise import KNNBasic,NormalPredictor,BaselineOnly #NMF



In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset
from surprise import Reader
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pandas as pd
import random
import difflib
from surprise.model_selection import cross_validate
from collections import defaultdict

from surprise import Dataset, SVD
from surprise.model_selection import KFold

In [3]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [4]:
import gdown

In [5]:
url = 'https://drive.google.com/uc?export=download&id=1tY2U8IjcUo2NTbXMWvY1Bg9eG8qn9fcK'
gdown.download(url, 'reviews.parquet', quiet=False)

# Ahora puedes cargar el archivo como un DataFrame de pandas o usarlo como lo necesites
import pandas as pd
reviews = pd.read_parquet('reviews.parquet',engine='pyarrow')

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1tY2U8IjcUo2NTbXMWvY1Bg9eG8qn9fcK
From (redirected): https://drive.google.com/uc?export=download&id=1tY2U8IjcUo2NTbXMWvY1Bg9eG8qn9fcK&confirm=t&uuid=6e48bc20-c200-4ba0-b7b2-2126ecaa94b5
To: c:\Users\maria\Documents\ITBA MARIA\funcionaplis_3\reviews.parquet
100%|██████████| 144M/144M [00:12<00:00, 11.1MB/s] 


In [6]:
#reviews = pd.read_parquet('reviews.parquet', engine='pyarrow')
print(reviews.columns)

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')


In [53]:
reviews = pd.read_parquet('reviews.parquet', engine='pyarrow')
reviews= reviews[['user_id', 'business_id', 'stars']]

In [43]:
print(reviews.columns)

Index(['user_id', 'business_id', 'stars'], dtype='object')


In [47]:
reviews.head(10)

,user_id,business_id,stars
0,nkN_do3fJ9xekchVC-v68A,8A5LSwsKK5vXDH1M3_Tf1w,5
1,nkN_do3fJ9xekchVC-v68A,U3grYFIeu6RgAAQgdriHww,5
2,nkN_do3fJ9xekchVC-v68A,UjOq8dBVNv9weHrVDqm3kQ,1
3,nkN_do3fJ9xekchVC-v68A,CoZ2mpsMBP8HUG1ymKoZTg,3
4,iUeZhkI0OK0BisakOkb3pQ,IZZ_uR2ps5KuuyE0n--jVA,5
5,iUeZhkI0OK0BisakOkb3pQ,LuB-RHZQzmbAHXmykrN0BA,3
6,iUeZhkI0OK0BisakOkb3pQ,ZTctPm8-lBy0iJ9dFhYhyQ,5
7,iUeZhkI0OK0BisakOkb3pQ,YxzjYHBkWLwxpvOlcjKkhw,4
8,iUeZhkI0OK0BisakOkb3pQ,g1xRzUV-aLjAE4mkdhLwBQ,5
9,nzsv-p1O8gCfP3XijfQrIw,QhxtQppYwOCKjExMftCa9Q,4


In [46]:
restaurant = pd.read_parquet('mi_archivo1.parquet', engine='pyarrow')
print(restaurant.columns)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')


In [50]:
restaurant.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '..."
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Food, Delis, Italian, Bakeries, Restaurants","{'Friday': '10:0-20:0', 'Monday': '10:0-18:0',..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Sushi Bars, Restaurants, Japanese","{'Friday': '13:30-23:0', 'Monday': None, 'Satu..."
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Korean, Restaurants","{'Friday': '11:30-20:30', 'Monday': '11:30-20:..."
23,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Italian","{'Friday': '11:0-21:0', 'Monday': '11:0-22:0',..."
27,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,Nashville,TN,37201,36.159886,-86.773197,4.0,161,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Japanese, Seafood","{'Friday': '16:0-23:0', 'Monday': None, 'Satur..."
33,kV_Q1oqis8Qli8dUoGpTyQ,Ardmore Pizza,10 Rittenhouse Pl,Ardmore,PA,19003,40.006707,-75.289671,3.5,109,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Pizza, Restaurants","{'Friday': '11:0-1:0', 'Monday': '11:0-0:0', '..."
41,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002,38.896563,-90.186203,3.0,94,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Specialty Food, Steakhouses, Food...","{'Friday': '16:0-22:0', 'Monday': '0:0-0:0', '..."
47,lk9IwjZXqUMqqOhM774DtQ,Caviar & Bananas,2031 Broadway,Nashville,TN,37203,36.148371,-86.798895,3.5,159,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Coffee & Tea, Restaurants, Wine Bars, Bars, Ni...","{'Friday': '7:0-17:0', 'Monday': '7:0-17:0', '..."
61,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,6880 E 82nd St,Indianapolis,IN,46250,39.906295,-86.047463,3.5,257,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Sports Bars, American (New), American (Traditi...","{'Friday': '11:0-19:0', 'Monday': '0:0-0:0', '..."


#### Modelo de SR

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews, reader) 
trainset, testset = train_test_split(data, test_size=0.2)  # El 20% para test
svd = SVD()
svd.fit(trainset) #Entreno un modelo con SVD 

In [60]:
import difflib
import random

def get_restaurant_id(restaurant_title, metadata):
    
    existing_titles = list(metadata['name'].values)
    closest_titles = difflib.get_close_matches(restaurant_title, existing_titles)
    restaurant_id = metadata[metadata['name'] == closest_titles[0]]['business_id'].values[0]
    return restaurant_id

def get_restaurant_info(restaurant_id, metadata):
    
    restaurant_info = metadata[metadata['business_id'] == restaurant_id][['business_id', 'latitude', 
                                                    'longitude', 'name', 'stars']]
    return restaurant_info.to_dict(orient='records')

def predict_review(user_id, restaurant_title, model, metadata):
    
    restaurant_id = get_restaurant_id(restaurant_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=restaurant_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=4):
    
    restaurant_titles = list(metadata['name'].values)
    random.shuffle(restaurant_titles)
    
    for restaurant_title in restaurant_titles:
        rating = predict_review(user_id, restaurant_title, model, metadata)
        if rating >= thresh:
            restaurant_id = get_restaurant_id(restaurant_title, metadata)
            return get_restaurant_info(restaurant_id, metadata)

In [64]:
#Hago la prediccion poniendo un user_id
user_id = 'nkN_do3fJ9xekchVC-v68A'
generate_recommendation(user_id, svd, restaurant, 4)#Umbral de que va a predecir que por lo menos puntea con 4 a este restaurante

[{'business_id': 'wbLXXbI-T6Av71i9AoiZRQ',
  'latitude': 38.8116095,
  'longitude': -89.9556965,
  'name': 'Cleveland-Heath',
  'stars': 4.5}]

#### Comparando otros algoritmos ademas de SVD a ver como funcionan

In [68]:
reader = Reader(rating_scale=(1, 5))  # Suponiendo que las estrellas están en el rango de 1 a 5
data = Dataset.load_from_df(reviews[['user_id', 'business_id', 'stars']], reader)

# Inicializar los algoritmos
algo_random = NormalPredictor()
algo_baseline = BaselineOnly()
algo_svd = SVD(random_state=100)
algo_knn = KNNBasic(random_state=100)

# Evaluar los algoritmos utilizando cross-validation
cv_random = cross_validate(algo_random, data, measures=['RMSE'], cv=4, verbose=False, n_jobs=-1)
cv_baseline = cross_validate(algo_baseline, data, measures=['RMSE'], cv=4, verbose=False, n_jobs=-1)
cv_svd = cross_validate(algo_svd, data, measures=['RMSE'], cv=4, verbose=False, n_jobs=-1)
cv_knn = cross_validate(algo_knn, data, measures=['RMSE'], cv=4, verbose=False, n_jobs=-1)

In [ ]:
res=[]
res.append(round(cv_random['test_rmse'].mean(),3))
res.append(round(cv_baseline['test_rmse'].mean(),3))
res.append(round(cv_svd['test_rmse'].mean(),3))
res.append(round(cv_knn['test_rmse'].mean(),3))


res_df=pd.DataFrame(pd.concat([pd.Series(['Random','Baseline','SVD','KNN']),pd.Series(res)],axis=1))
res_df.columns=['Algoritmo','RMSE CV']
res_df.sort_values(by='RMSE CV')

#El modelo que mejor funciona es el Baseline, de todas maneras elegimos continuar con el SVD

,Algoritmo,RMSE CV
1,Baseline,0.879
2,SVD,0.887
3,KNN,0.971
0,Random,1.306


#### Teniendo en cuenta metricas para los sistemas de recomendacion

**RELEVANTE** --> Puntuacion es mayor al umbral definido 

**RECOMENDADO** --> Lo mismo que relevante + se encuentra entre las k mejores estimadas


*PRECISSION@KEY* --> Items recomendados que son relevantes (de los que catalogo como + cuantos son positivos realmente)

*RECALL@KEY* --> Proporcion de items relevantes que son recomendados (de los positivos reales cuales estan siendo recomendados)

In [73]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [74]:
#Pruebo con 5 recomendaciones a ver como me va 
kf = KFold(n_splits=4)
algo = SVD(n_epochs=10, random_state=100)
precision_5=[]
recall_5=[]

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)
    precision_5.append(sum(prec for prec in precisions.values()) / len(precisions))
    recall_5.append(sum(rec for rec in recalls.values()) / len(recalls))

#Top 5
print(f'Precision @5 medio: {round(np.array(precision_5).mean(),2)}')
print(f'Recall @5 medio: {round(np.array(recall_5).mean(),2)}')

Precision @5 medio: 0.59
Recall @5 medio: 0.39


Esto significa que  en promedio, 

RECALL --> el 59% del top 5 de recomendaciones hechas para cada usuario son relevantes (con un puntaje estimado mayor o igual a 4)

PRECISSION --> mientras que de todos los restaurantes relevantes, el 39% aparecieron en el top 5.

Reducir k es mayor precision pero menor recall --> Igual busco mayor precision porque no quieroo pifiarle a los restaurantes recomendados. Por eso pruebo reducir k 

**Thredshold** --> A partir de mayor a que puntuacion lo recomienda

In [78]:

#Pruebo con 3 recomendaciones a ver como me va 
kf = KFold(n_splits=4)
algo = SVD(n_epochs=10, random_state=100)
precision_3=[]
recall_3=[]

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    
    precisions, recalls = precision_recall_at_k(predictions, k= 3, threshold=4)
    precision_3.append(sum(prec for prec in precisions.values()) / len(precisions))
    recall_3.append(sum(rec for rec in recalls.values()) / len(recalls))

#Top 3
print(f'Precision @3 medio: {round(np.array(precision_3).mean(),2)}')
print(f'Recall @3 medio: {round(np.array(recall_3).mean(),2)}')

Precision @3 medio: 0.59
Recall @3 medio: 0.35


No cambia mucho disminuyendo k entonces conservamos un k = 5

In [79]:
#Serializo el modelo para poder usarlo en una app de Streamlit 
import pickle

# Guarda el modelo serializado
with open('./Data/svd_model.pkl', 'wb') as model_file:
    pickle.dump(svd, model_file)
print("Modelo SVD serializado como 'svd_model.pkl'")

Modelo SVD serializado como 'svd_model.pkl'
